https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [1]:
import pandas as pd
data = pd.read_csv('file2_lily.csv',error_bad_lines=False);
data = data.dropna(how='any',axis=0)
data_text = data[['Description']]
data_text['index'] = data_text.index
documents = data_text

/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [2]:
documents[:10]

,Description,index
1,"Part road-trip tale, part travelogue of lost ...",1
2,If you're making the leap from SAP BW to SAP ...,2
4,The first collection of poetry by the New Yor...,4
6,"""This is a compassionate look at poverty, har...",6
7,Barrie Watson has been a virtual shut-in all ...,7
8,Imagine losing a loved one in the public eye....,8
9,The Gospel of John is the most deeply spiritu...,9
10,There once were seven very BAD cats...Seven c...,10
17,A New York Times BestsellerGet the recipes ev...,17
18,Bruce the bear likes to keep to himself. That...,18


In [3]:
documents['Description'].iloc[0]

' Part road-trip tale, part travelogue of lost and found landscapes, all good-natured natural history, Mariposa Road tracks Bob Pyle’s journey across the United States as he races against the calendarin his search for as many of the 800 American butterflies as he can find.\xa0Like Pyle’s classic Chasing Monarchs, Mariposa Road recounts his adventures, high and low, in tracking down butterflies in his own low-tech, individual way. Accompanied by Marsha, his cottonwood-limb butterfly net; Powdermilk, his 1982 Honda Civic with 345,000 miles on the odometer; and the small Leitz binoculars he has carried for more than thirty years, Bob ventured out in a series of remarkable trips from his Northwest home.\xa0From the California coastline in company with overwintering monarchs to the Far Northern tundra in pursuit of mysterious sulphurs and arctics; from the zebras and daggerwings of the Everglades to the leafwings, bluewings, and border rarities of the lower Rio Grande; from Graceland to ran

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/LovelyCheuk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
doc_sample = documents[documents['index'] == 1].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'Part', 'road-trip', 'tale,', 'part', 'travelogue', 'of', 'lost', 'and', 'found', 'landscapes,', 'all', 'good-natured', 'natural', 'history,', 'Mariposa', 'Road', 'tracks', 'Bob', 'Pyle’s', 'journey', 'across', 'the', 'United', 'States', 'as', 'he', 'races', 'against', 'the', 'calendarin', 'his', 'search', 'for', 'as', 'many', 'of', 'the', '800', 'American', 'butterflies', 'as', 'he', 'can', 'find.\xa0Like', 'Pyle’s', 'classic', 'Chasing', 'Monarchs,', 'Mariposa', 'Road', 'recounts', 'his', 'adventures,', 'high', 'and', 'low,', 'in', 'tracking', 'down', 'butterflies', 'in', 'his', 'own', 'low-tech,', 'individual', 'way.', 'Accompanied', 'by', 'Marsha,', 'his', 'cottonwood-limb', 'butterfly', 'net;', 'Powdermilk,', 'his', '1982', 'Honda', 'Civic', 'with', '345,000', 'miles', 'on', 'the', 'odometer;', 'and', 'the', 'small', 'Leitz', 'binoculars', 'he', 'has', 'carried', 'for', 'more', 'than', 'thirty', 'years,', 'Bob', 'ventured', 'out', 'in', 'a', 'series', 'of'

In [7]:
processed_docs = documents['Description'].map(preprocess)

In [8]:
processed_docs = documents['Description'].map(preprocess)
processed_docs[:10]

1     [road, trip, tale, travelogu, lose, landscap, ...
2     [make, leap, hana, book, indispens, companion,...
4     [collect, poetri, york, time, bestsel, author,...
6     [compassion, look, poverti, hard, choic, defen...
7     [barri, watson, virtual, shut, life, move, sou...
8     [imagin, lose, love, public, media, frenzi, en...
9     [gospel, john, deepli, spiritu, gospel, write,...
10    [seven, cat, seven, cat, probabl, seven, cat, ...
17    [york, time, bestsellerget, recip, talk, handi...
18    [bruce, bear, like, egg, hard, boil, goos, egg...
Name: Description, dtype: object

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [10]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accompani
1 adventur
2 american
3 arctic
4 binocular
5 bluew
6 border
7 butterfli
8 calendarin
9 california
10 carri


In [11]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1)]

In [15]:
bow_doc_0 = bow_corpus[0]

for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                                     dictionary[bow_doc_0[i][0]], 
                                                     bow_doc_0[i][1]))

Word 0 ("adventur") appears 1 time.
Word 1 ("american") appears 1 time.
Word 2 ("classic") appears 1 time.
Word 3 ("countri") appears 1 time.
Word 4 ("extraordinari") appears 1 time.
Word 5 ("good") appears 1 time.
Word 6 ("heart") appears 1 time.
Word 7 ("high") appears 1 time.
Word 8 ("histori") appears 1 time.
Word 9 ("home") appears 1 time.
Word 10 ("inspir") appears 1 time.
Word 11 ("journey") appears 2 time.
Word 12 ("lead") appears 1 time.
Word 13 ("like") appears 1 time.
Word 14 ("lose") appears 1 time.
Word 15 ("mysteri") appears 1 time.
Word 16 ("natur") appears 2 time.
Word 17 ("origin") appears 1 time.
Word 18 ("peopl") appears 1 time.
Word 19 ("reader") appears 1 time.
Word 20 ("search") appears 1 time.
Word 21 ("seri") appears 1 time.
Word 22 ("small") appears 1 time.
Word 23 ("state") appears 1 time.
Word 24 ("tale") appears 1 time.
Word 25 ("turn") appears 1 time.
Word 26 ("year") appears 1 time.


In [16]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [17]:
corpus_tfidf = tfidf[bow_corpus]

In [18]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.1896817565175649),
 (1, 0.13616465623012936),
 (2, 0.1815469917216657),
 (3, 0.1777799590017366),
 (4, 0.19877564986109572),
 (5, 0.1777799590017366),
 (6, 0.1777799590017366),
 (7, 0.17075613215210664),
 (8, 0.14770983022422038),
 (9, 0.15837625970208172),
 (10, 0.13616465623012936),
 (11, 0.3793635130351298),
 (12, 0.1896817565175649),
 (13, 0.11893599809069934),
 (14, 0.1777799590017366),
 (15, 0.1940948922144773),
 (16, 0.3793635130351298),
 (17, 0.16747015304561255),
 (18, 0.13616465623012936),
 (19, 0.13616465623012936),
 (20, 0.20375859519361803),
 (21, 0.18550728826012086),
 (22, 0.1896817565175649),
 (23, 0.1815469917216657),
 (24, 0.1815469917216657),
 (25, 0.16129014722003585),
 (26, 0.08593353504918798)]


In [19]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.059*"time" + 0.030*"book" + 0.021*"stori" + 0.018*"york" + 0.018*"life" + 0.018*"author" + 0.018*"busi" + 0.017*"work" + 0.016*"today" + 0.015*"includ"
Topic: 1 
Words: 0.053*"busi" + 0.036*"book" + 0.031*"chang" + 0.023*"help" + 0.023*"start" + 0.021*"learn" + 0.020*"creat" + 0.019*"plan" + 0.019*"know" + 0.019*"histori"
Topic: 2 
Words: 0.050*"famili" + 0.036*"love" + 0.028*"stori" + 0.028*"live" + 0.022*"murder" + 0.019*"countri" + 0.019*"novel" + 0.018*"like" + 0.017*"life" + 0.017*"world"
Topic: 3 
Words: 0.048*"life" + 0.031*"time" + 0.024*"book" + 0.021*"world" + 0.020*"year" + 0.020*"stori" + 0.018*"famili" + 0.017*"best" + 0.016*"love" + 0.015*"york"
Topic: 4 
Words: 0.073*"stori" + 0.032*"write" + 0.025*"live" + 0.024*"american" + 0.023*"town" + 0.023*"take" + 0.022*"make" + 0.020*"collect" + 0.017*"time" + 0.017*"like"
Topic: 5 
Words: 0.038*"human" + 0.037*"book" + 0.034*"world" + 0.034*"seri" + 0.020*"know" + 0.020*"time" + 0.018*"year" + 0.017*"stori" +

In [21]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [22]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.023*"busi" + 0.021*"plan" + 0.019*"build" + 0.016*"guid" + 0.016*"live" + 0.014*"famili" + 0.013*"want" + 0.013*"help" + 0.013*"start" + 0.013*"york"
Topic: 1 Word: 0.038*"citi" + 0.022*"word" + 0.019*"grow" + 0.019*"murder" + 0.018*"figur" + 0.016*"seri" + 0.016*"illustr" + 0.015*"york" + 0.014*"year" + 0.013*"mysteri"
Topic: 2 Word: 0.022*"beauti" + 0.018*"home" + 0.017*"like" + 0.017*"busi" + 0.016*"popular" + 0.015*"chang" + 0.015*"adventur" + 0.015*"discov" + 0.014*"hand" + 0.014*"grow"
Topic: 3 Word: 0.017*"town" + 0.016*"world" + 0.016*"famili" + 0.016*"life" + 0.014*"kind" + 0.014*"countri" + 0.013*"mysteri" + 0.013*"live" + 0.012*"year" + 0.012*"past"
Topic: 4 Word: 0.019*"life" + 0.019*"explor" + 0.017*"young" + 0.017*"scienc" + 0.016*"book" + 0.016*"best" + 0.015*"love" + 0.014*"novel" + 0.014*"experi" + 0.013*"world"
Topic: 5 Word: 0.042*"book" + 0.037*"classic" + 0.021*"perfect" + 0.019*"hous" + 0.018*"self" + 0.017*"friend" + 0.016*"come" + 0.015*"human" 

In [24]:
processed_docs[1]

['road',
 'trip',
 'tale',
 'travelogu',
 'lose',
 'landscap',
 'good',
 'natur',
 'natur',
 'histori',
 'mariposa',
 'road',
 'track',
 'pyle',
 'journey',
 'unit',
 'state',
 'race',
 'calendarin',
 'search',
 'american',
 'butterfli',
 'like',
 'pyle',
 'classic',
 'chase',
 'monarch',
 'mariposa',
 'road',
 'recount',
 'adventur',
 'high',
 'track',
 'butterfli',
 'tech',
 'individu',
 'accompani',
 'marsha',
 'cottonwood',
 'limb',
 'butterfli',
 'powdermilk',
 'honda',
 'civic',
 'mile',
 'odomet',
 'small',
 'leitz',
 'binocular',
 'carri',
 'thirti',
 'year',
 'ventur',
 'seri',
 'remark',
 'trip',
 'northwest',
 'home',
 'california',
 'coastlin',
 'compani',
 'overwint',
 'monarch',
 'northern',
 'tundra',
 'pursuit',
 'mysteri',
 'sulphur',
 'arctic',
 'zebra',
 'daggerw',
 'everglad',
 'leafw',
 'bluew',
 'border',
 'rariti',
 'lower',
 'grand',
 'graceland',
 'ranchland',
 'kauai',
 'west',
 'intim',
 'encount',
 'land',
 'peopl',
 'fade',
 'fauna',
 'wholli',
 'origin',
 

In [25]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5001440644264221	 
Topic: 0.053*"busi" + 0.036*"book" + 0.031*"chang" + 0.023*"help" + 0.023*"start" + 0.021*"learn" + 0.020*"creat" + 0.019*"plan" + 0.019*"know" + 0.019*"histori"

Score: 0.4527895450592041	 
Topic: 0.032*"book" + 0.029*"famili" + 0.027*"know" + 0.024*"help" + 0.023*"live" + 0.019*"break" + 0.018*"come" + 0.017*"discov" + 0.016*"learn" + 0.016*"find"


In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5549411773681641	 
Topic: 0.027*"murder" + 0.020*"learn" + 0.019*"polit" + 0.019*"seri" + 0.017*"know" + 0.015*"star" + 0.014*"struggl" + 0.014*"origin" + 0.013*"detail" + 0.013*"human"

Score: 0.2296331375837326	 
Topic: 0.019*"life" + 0.019*"explor" + 0.017*"young" + 0.017*"scienc" + 0.016*"book" + 0.016*"best" + 0.015*"love" + 0.014*"novel" + 0.014*"experi" + 0.013*"world"

Score: 0.17424091696739197	 
Topic: 0.032*"american" + 0.022*"reader" + 0.018*"stori" + 0.017*"includ" + 0.016*"tradit" + 0.015*"like" + 0.015*"world" + 0.014*"america" + 0.013*"book" + 0.013*"lead"
